In [86]:
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
import os
from sqlite3 import Error

In [87]:
df = pd.read_csv("Sample-Superstore.csv", encoding='latin-1')

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [89]:
# df['Month'] = df['Order Date'].dt.month
# df['Year'] = df['Order Date'].dt.year

In [90]:
df = df.drop(columns=['Row ID'])
df = df.rename(columns={'Order ID': 'OrderID', 'Customer ID': 'CustomerID', 'Product ID': 'ProductID'})

In [91]:
# Get unique categories
unique_categories = df['Category'].unique()

# Create a mapping of category names to IDs
category_id_mapping = {category: idx + 1 for idx, category in enumerate(unique_categories)}

# Add a Category ID column to the DataFrame
df['CategoryID'] = df['Category'].map(category_id_mapping)
df.head(2)

,OrderID,Order Date,Ship Date,Ship Mode,CustomerID,Customer Name,Segment,Country,City,State,...,Region,ProductID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,CategoryID
0,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,1
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820,1


In [92]:
df_Address = df[["Country", "City", "State", "Postal Code", "Region"]]
df['AddressID'] = range(1, len(df_Address) + 1)

In [93]:
import pandas as pd
import hashlib

def create_address_id(df):
    df['address_string'] = df['City'].astype(str) + df['State'].astype(str) + df['Postal Code'].astype(str) + df['Country'].astype(str)
    print(df['address_string'].isnull().sum())  # Check for NaN
    print(df['address_string'].head())  # Inspect data

    def hash_address(address):
        if pd.isna(address):
            return None
        return hashlib.md5(address.encode()).hexdigest()

    df['AddressID'] = df['address_string'].apply(hash_address)
    df = df.drop(columns=['address_string'])
    return df


In [94]:
df.head(2)

,OrderID,Order Date,Ship Date,Ship Mode,CustomerID,Customer Name,Segment,Country,City,State,...,ProductID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,CategoryID,AddressID
0,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,1,1
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820,1,2


In [95]:
dfC = sqlite3.connect("superstore_database.db")

In [96]:
c = dfC.cursor()

In [97]:
c.execute('''
CREATE TABLE IF NOT EXISTS "Customer" (
	"CustomerID"	CHAR(30) NOT NULL,
	"Customer Name"	VARCHAR(30) NOT NULL,
	"Segment"	VARCHAR(15) NOT NULL,
	PRIMARY KEY("CustomerID")
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Address" (
	"Address_ID" INTEGER PRIMARY KEY AUTOINCREMENT,
    "Country" text NOT NULL,
    "City" text NOT NULL,
    "State" text NOT NULL,
    "Postal Code" integer NOT NULL,
    "Region" text NOT NULL
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Product" (
	"ProductID"	CHAR(20) NOT NULL,
    "Product Name"	VARCHAR(30) NOT NULL,
    "CategoryID"	VARCHAR(30) NOT NULL,
	PRIMARY KEY("ProductID")
	FOREIGN KEY(CategoryID) REFERENCES Product_Category(CategoryID)
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Category" (
	"CategoryID" CHAR(20) NOT NULL,
	"Category" VARCHAR(30) NOT NULL,
	"Sub-Category" VARCHAR(30) NOT NULL,
	PRIMARY KEY("CategoryID")
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Order" (
	"OrderID" CHAR(20) NOT NULL,
	"Order Date" date NOT NULL,
    "Ship Date" date NOT NULL, 
    "Ship Mode" varchar NOT NULL,
    "CustomerID" varchar NOT NULL,
	PRIMARY KEY("OrderID"),
	FOREIGN KEY(CustomerID) REFERENCES Customer_Details(Customer_ID)
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Sales" (
        "Sales" integer NOT NULL,
    "Quantity"  integer NOT NULL,
    "Discount"  integer NOT NULL,
    "Profit"  integer NOT NULL,
    "OrderID" INTEGER NOT NULL,
    "ProductID" INTEGER NOT NULL,
    PRIMARY KEY("OrderID", "ProductID"),
    FOREIGN KEY(OrderID) REFERENCES Order_Details(OrderID),
    FOREIGN KEY(ProductID) REFERENCES Product_Details(ProductID)
);
''')


In [98]:
df['UnitPrice'] = df['Sales'] / (df['Quantity'] * (1-df['Discount']))
df['BuyPrice'] = df['UnitPrice'] - df['Profit']
df['OrderID_ProductID'] = df['OrderID'] + ' ' + df['ProductID']
df.info()

df_Sales = df[["Sales", "Quantity", "Discount", "Profit", "OrderID", "ProductID"] ]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   OrderID            9994 non-null   object 
 1   Order Date         9994 non-null   object 
 2   Ship Date          9994 non-null   object 
 3   Ship Mode          9994 non-null   object 
 4   CustomerID         9994 non-null   object 
 5   Customer Name      9994 non-null   object 
 6   Segment            9994 non-null   object 
 7   Country            9994 non-null   object 
 8   City               9994 non-null   object 
 9   State              9994 non-null   object 
 10  Postal Code        9994 non-null   int64  
 11  Region             9994 non-null   object 
 12  ProductID          9994 non-null   object 
 13  Category           9994 non-null   object 
 14  Sub-Category       9994 non-null   object 
 15  Product Name       9994 non-null   object 
 16  Sales              9994 

In [99]:
df_Sales.to_sql('Sales', dfC, if_exists='replace', index=False)
dfC.commit()

In [100]:
df_order = df[["OrderID", "Order Date", "Ship Date", "Ship Mode", "CustomerID"]]

In [101]:
df_order = df_order.drop_duplicates(subset=['OrderID'])

In [102]:
df_order.to_sql('Order', dfC, if_exists='replace', index=False)

5009

In [103]:
df_Customer = df[["CustomerID", "Customer Name", "Segment"]]
df_Customer = df_Customer.drop_duplicates(subset=['CustomerID'])
df_Customer.to_sql('Customer', dfC, if_exists='replace', index=False)

793

In [104]:
df_Address = df[["Country", "City", "State", "Postal Code", "Region"]]
df_Address = df_Address.drop_duplicates()
df_Address.to_sql('Address', dfC, if_exists='replace', index=False)

632

In [105]:
df_Category = df[['CategoryID' ,'Category', 'Sub-Category']]
df_Category = df_Category.drop_duplicates()
df_Category.to_sql('Category', dfC, if_exists='replace', index=False)

17

In [106]:
df_Product = df[["ProductID", "Product Name", "CategoryID"]]
df_Product = df_Product.drop_duplicates(subset=['ProductID'])
df_Product.to_sql('Product', dfC, if_exists='replace', index=False)

1862

In [107]:
# Adding a sequential number column
# df_Address['AddressID'] = range(1, len(df_Address) + 1)
# df_Address['Full Address'] = df['Country'] + ' ' + df['City'].fillna('') + ' ' + df['State'] + ' ' + df['Postal Code'].fillna('').astype(str) + ' ' + df['Region']
# df_Address = df_Address.drop(["Country", "City", "State", "Postal Code", "Region"],axis=1)
# df_Address.tail()
# df_Address.info()

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   OrderID            9994 non-null   object 
 1   Order Date         9994 non-null   object 
 2   Ship Date          9994 non-null   object 
 3   Ship Mode          9994 non-null   object 
 4   CustomerID         9994 non-null   object 
 5   Customer Name      9994 non-null   object 
 6   Segment            9994 non-null   object 
 7   Country            9994 non-null   object 
 8   City               9994 non-null   object 
 9   State              9994 non-null   object 
 10  Postal Code        9994 non-null   int64  
 11  Region             9994 non-null   object 
 12  ProductID          9994 non-null   object 
 13  Category           9994 non-null   object 
 14  Sub-Category       9994 non-null   object 
 15  Product Name       9994 non-null   object 
 16  Sales              9994 

In [109]:
df.columns

Index(['OrderID', 'Order Date', 'Ship Date', 'Ship Mode', 'CustomerID',
       'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code',
       'Region', 'ProductID', 'Category', 'Sub-Category', 'Product Name',
       'Sales', 'Quantity', 'Discount', 'Profit', 'CategoryID', 'AddressID',
       'UnitPrice', 'BuyPrice', 'OrderID_ProductID'],
      dtype='object')

In [110]:
# df_order = df[["OrderID", "Order Date", "Ship Date", "Ship Mode", "CustomerID"]]
# df_order = df_order.drop_duplicates(subset=['OrderID'])
# df_order.to_sql('Order', dfC, if_exists='append', index=False)

# df_Customer = df[["CustomerID", "Customer Name", "Segment"]]
# df_Customer = df_Customer.drop_duplicates(subset=['CustomerID'])
# df_Customer.to_sql('Customer', dfC, if_exists='append', index=False)

# df_Address = df[["Country", "City", "State", "Postal Code", "Region"]]
# df_Address = df_Address.drop_duplicates()
# df_Address.to_sql('Address', dfC, if_exists='append', index=False)

# df_Category = df[['CategoryID' ,'Category', 'Sub-Category']]
# df_Category = df_Category.drop_duplicates()
# df_Category.to_sql('Category', dfC, if_exists='append', index=False)

# df_Product = df[["ProductID", "Product Name", "CategoryID"]]
# df_Product = df_Product.drop_duplicates(subset=['ProductID'])
# df_Product.to_sql('Product', dfC, if_exists='append', index=False)

#Commit changes
dfC.commit() 


In [111]:
df.columns

Index(['OrderID', 'Order Date', 'Ship Date', 'Ship Mode', 'CustomerID',
       'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code',
       'Region', 'ProductID', 'Category', 'Sub-Category', 'Product Name',
       'Sales', 'Quantity', 'Discount', 'Profit', 'CategoryID', 'AddressID',
       'UnitPrice', 'BuyPrice', 'OrderID_ProductID'],
      dtype='object')

# Epic 4: Perform some queries

# What is the category generating the maximum sales revenue?
# What about the profit in this category?
# Are they making a loss in any categories?